In [12]:
from pymongo import MongoClient
from bson.json_util import dumps
import pymongo
import pandas as pd
import requests

mongodbURL = "mongodb://localhost/Chat_Sentiment_Analysis_Service"
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

We are going to use 

In [13]:
shakespeare=pd.read_csv("input/Shakespeare_data.csv").drop(columns=["PlayerLinenumber","ActSceneLine","Dataline"],axis=1).dropna()

shakespeare["Play"]=shakespeare["Play"].apply(lambda x: x.replace(" ",""))
shakespeare["Player"]=shakespeare["Player"].apply(lambda x: x.replace(" ",""))
shakespeare["PlayerLine"]=shakespeare["PlayerLine"].apply(lambda x: x.replace(" ","%20"))



shakespeare_plays=shakespeare["Play"].unique()

shakespeare_players=shakespeare["Player"].unique()

players_in_play=[{play:shakespeare[shakespeare["Play"]==play]["Player"].unique()} for play in shakespeare_plays]

Feeding users database

In [14]:
for name in shakespeare_players:
    requests.get(f"http://127.0.0.1:4000/user/create?username={name}").text

Creating conversations (plays)

In [15]:
for play in shakespeare_plays:
    requests.get(f"http://127.0.0.1:4000/chat/create?conversation={play}").text

Adding each character to its play

In [17]:
for play in players_in_play:
        for k,v in play.items():
            for player in v:
                requests.get(f"http://127.0.0.1:4000/chat/{k}/adduser?username={player}")

Adding sentence of each character to its play

In [18]:
for index in shakespeare.index:
    requests.get(f"http://127.0.0.1:4000/chat/{shakespeare.loc[index]['Play']}/addmessage?username={shakespeare.loc[index]['Player']}&message={shakespeare.loc[index]['PlayerLine']}")

With the code beloew you can clear the database just in case something is corrupted.

db.messages.remove({})
db.users.remove({})
db.conversations.remove({})

In [4]:
db.messages.remove({})
db.users.remove({})
db.conversations.remove({})

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until


{'n': 38, 'ok': 1.0}